# Harmonized Landsat Sentinel (HLS) EVI extraction

**Project:** Georgia and Texas Agriculture

**Date:** 03/10/2025

**Code Contact:** Henry Osei, henryoseipoku77@gmail.com

**Inputs:** Boundary shapefile for each NASS district of interest. Cotton mask for each district that was generated in the TXGA_CottonMask.ipynb file. 

**Outputs:** CSV files of monthly Enhanced Vegetation Index (EVI).

**Description:** This script derives EVI values per month (March to November) for the NASS districts of interest from 2015 to 2024. Firstly, the red, blue, and NIR bands are extracted from the HLS data to calculate EVI. EVI tiles for each district are mosaicked into a single EVI tile per district. Half monthly composites of EVI are then created for a finer EVI time series. Finally, all half monthly EVI csv files per year are assembled together to produce montly EVI per district.   
- NASS districts of interest: **Georgia**: GA-70, GA-80, GA-90; **Texas**: TX-12, TX-21, TX-22, TX-60, TX-70

In [ ]:
# Import the necessary libraries
import dask.array as da
import os
import pandas as pd
import geopandas as gpd
import numpy as np
import rasterio
from rasterio.merge import merge
from rasterio.warp import reproject, Resampling
from rasterio.transform import from_origin
import rioxarray as rxr
import earthaccess
from dask.diagnostics import ProgressBar
import dask.dataframe as dd
import dask
import dask.delayed
from rasterio.plot import show
from rasterio.crs import CRS
import xarray as xr
import glob

### PART A
- This is where we will calculate EVI and create csv files of half monthly EVI composites. 
 
**NB:** The processing in this section is computationally demanding, so unless you are working on a 'supercomputer' (RAM >16GB, speed >3.0, CPUs> 12, etc.), it is recommended that you do this section per district as was done here. 

In [ ]:
# base directory for all the input files
# NB: On your computer, change this to the directory of the input files 
path= os.chdir('C:/TX_GA_CSB/per_district')

# create a folder to store the output files
int_dir = "Image_files" # one to store the intermediate output files
os.makedirs(int_dir)

final_dir = "EVI_CSV" # another to store the final output files
os.makedirs(final_dir)

In [ ]:
# read the GA_90 district boundary and cotton mask shapefiles
GA_90 = gpd.read_file('GA_90.shp')
GA_90_farms = gpd.read_parquet('GA_90_CottonMask.geoparquet')


# we will need the bounding box to extract all HLS data that intersects with the district, so
# extract the bounding box for GA_90 district
GA_90bbox = tuple(GA_90.total_bounds)

In [ ]:
#NB: before you can use the earthaccess python API to extract HLS data, you need to have an Earthdata account
# authenticate earthaccess
earthaccess.login()

**NB:** The HLS dataset can provide data every to 2 to 3 days (starting from 2018). Also, one district can have up to 8 different tiles per acquisition date. Lastly, we need three different bands per tile to calculate EVI, meaning that we will be accessing a huge amount of raster files to calculate EVI at the district level.

So, if you are not using a supercomputer, it is recommended that you break the HLS data retrieval and processing into quarterly increments per year, as was done in this code. You can however retrieve data for all the desired months in one search for the year 2015, as Sentinel-2 data was not available until November 2015.   

In [ ]:
# --------------------------------
# use earthaccess to search and open all available HLS data that intersects with the GA_90 district in 2015, in quarterly timesteps
# --------------------------------

# data from Landsat 8/9, HLSL30
L30_results = earthaccess.search_data(
    short_name='HLSL30',
    cloud_hosted=True,
    temporal=("2015-03-01", "2015-05-31"), # March to May
    bounding_box=GA_90bbox,
)

# data from Sentinel-2, HLSS30
S30_results = earthaccess.search_data(
    short_name='HLSS30',
    cloud_hosted=True,
    temporal=("2015-03-01", "2015-05-31"),
    bounding_box=GA_90bbox,
)

# Combine S30 and L30 products into a time series
HLS_items = L30_results + S30_results
# len(HLS_items) #check the number of tiles in the specified timeframe

# define EVI function
def calc_EVI(red, blue, nir):
      return 2.5 * (nir - red) / (nir + 6.0 * red - 7.5 * blue + 1.0) 

**NB:** Some of the districts are located in more than one UTM coordinate system, so for a district, you can have some image tiles in UTM zone 13 and others in UTM zone 14. To accurately mosaic individual tiles of a district, we need to reproject all the image tiles into one coordinate system (WGS84). 

In [ ]:
# --------------------------------
# define a function to extract the nir, red, and blue bands of the HLS data.
# And for each acquisition date image composite (nir, red, blue bands), 
# calculate EVI and reproject the current coordinate reference system (crs) of the EVI to WGS84.
# --------------------------------


# crs to use later for reprojection of EVI images
wgs84_crs= CRS.from_epsg(4326)

# Define a function to process the nir, red, and blue bands for each acquisiton date granule into an EVI band and save on disk.
def process_granule_to_disk(gran):
    # Determine product type (HLSL30 or HLSS30) and set desired spectral bands accordingly
    if gran.get('umm')['RelatedUrls'][0]['URL'].split('/')[4] == 'HLSS30.020':
        desired_bands = ['B8A', 'B04', 'B02']  # Bands for HLSS30 product
    else:
        desired_bands = ['B05', 'B04', 'B02']  # Bands for HLSL30 product

    # extract urls for the desired bands, excluding 's3://' urls (we don't want the urls on amazon S3)
    band_urls = [
        url['URL']
        for url in gran.get('umm')['RelatedUrls']
        if any(band in url['URL'] for band in desired_bands) and 's3://' not in url['URL']
    ]
 
    # use earthaccess to create vsicurls so that we can stream the data (desired_bands) without downloading it onto our computer
    vsicurls = earthaccess.open(band_urls)

    # initialize arrays to store the band data and crs information
    # NB: we will need the crs of the current granule before we can reproject the crs of EVI 
    nir_array = red_array = blue_array = None
    original_crs = None

    # open and read the bands vsicurl into memory, apply scale factor and replace nodata values with nan
    for url in vsicurls:
        band_type = str(url).rsplit('.', 2)[-2] # extract the band number
        with rasterio.open(url) as src:
            band_data = src.read(1)
            band_data = np.where(band_data == src.nodata, np.nan, band_data)  # replace nodata values with nan
            band_scaled = da.from_array(band_data * src.scales[0], chunks=(500, 500))  # scale data. Use dask to break the data into chunks for fast processing

            # assign data to respective arrays based on band type
            if band_type in ('B8A', 'B05'):
                nir_array = band_data
                nir_scaled = band_scaled
            elif band_type == 'B04':
                red_array = band_data
                red_scaled = band_scaled
            elif band_type == 'B02':
                blue_array = band_data
                blue_scaled = band_scaled

            # extract the crs information from the nir band
            if original_crs is None:
                original_crs = src.crs

    # compute the EVI using scaled band data
    nir_scaled, red_scaled, blue_scaled = dask.compute(nir_scaled, red_scaled, blue_scaled)
    evi_band = calc_EVI(red_scaled, blue_scaled, nir_scaled)

    # reproject the EVI data to WGS84
    transform, width, height = rasterio.warp.calculate_default_transform(
        original_crs, wgs84_crs, evi_band.shape[1], evi_band.shape[0], *src.bounds
    )
    evi_reprojected = np.empty_like(evi_band)
    reproject(
        source=evi_band,
        destination=evi_reprojected,
        src_transform=src.transform,
        src_crs=original_crs,
        dst_transform=transform,
        dst_crs=wgs84_crs,
        resampling=Resampling.nearest
    )

    # Save the processed EVI data to disk as a GeoTIFF file
    gran_name = str(url).rsplit('/', 1)[-1][:-1]  # extract the product name from the band url
    out_name = os.path.join(int_dir,f"{gran_name.split('.B')[0]}_EVI.tif") # select the necessary name parts, add 'EVI' to the name, and save to the 'Image_files directory

    with rasterio.open(out_name, 'w', driver='GTiff', height=evi_reprojected.shape[0], 
                       width=evi_reprojected.shape[1], count=1, dtype=evi_reprojected.dtype, 
                       crs=wgs84_crs, transform=transform) as dst:
        dst.write(evi_reprojected, 1)

    return out_name

# process all granules in parallel using Dask
results = dask.compute(*[dask.delayed(process_granule_to_disk)(gran) for gran in HLS_items])

In [ ]:
# --------------------------------
# groups the file names based on acquisition date, mosaic them and clip them to the bounding box of the study area
# --------------------------------

# change the base directory to the Image_files folder
path= os.chdir('C:/TX_GA_CSB/per_district/Image_files')

# read all file names with '_EVI.tif' into a list
evi_files = glob.glob('*_EVI.tif')

# group the EVI filenames by Julian day (day of year, DOY)
date_groups = {}
for file in evi_files:
    j_date = int(file.split('.')[3][4:7])  # extract DOY from file name
    if j_date not in date_groups:
        date_groups[j_date] = []  # initialize group if DOY not seen before
    date_groups[j_date].append(file)

# mosaic grouped EVI files for each DOY and save as GeoTIFF
for date, filepaths in date_groups.items():
    # open the raster files for the current group
    sources = [rasterio.open(path) for path in filepaths]

    # create a mosaic
    mosaic, out_trans = merge(sources)

    # close the raster files 
    for src in sources:
        src.close()

    # define file name for the output mosaic using the acquisition date (DOY)
    mosaic_file = f"mosaic_{date}.tif"

    # update metadata for the mosaic file
    mosaic_meta = sources[0].meta.copy()
    mosaic_meta.update({
        "driver": "GTiff",
        "height": mosaic.shape[1],
        "width": mosaic.shape[2],
        "transform": out_trans,
        "dtype": mosaic.dtype
    })

    # save the mosaic as a  GeoTIFF file
    with rasterio.open(mosaic_file, "w", **mosaic_meta) as dst:
        dst.write(mosaic)


# read all file names with 'mosaic.tif' into a list
mosaic_files = glob.glob('mosaic*.tif')

# clip the mosaics to the bounding box of GA_90 and store the results
clipped_evis = {}  # dictionary to store the clipped EVIs
for date, file in zip(list(date_groups.keys()), mosaic_files):
        mosaic_rxr = rxr.open_rasterio(file, masked=True) # open the mosaic raster with rioxarray

        # clip the mosaic using the bbox of GA_90
        mosaic_clipped = mosaic_rxr.rio.clip_box(*GA_90.total_bounds)

        # some of the mosaicked EVIs does not cover the entire bbox of GA_90, so    
        # replace no-data values with NaN
        mosaic_clipped = mosaic_clipped.where(mosaic_clipped != 0, float("nan"))

        # EVI values can sometimes go beyond 1 when there are very bright objects, so 
        # mask values outside the valid range (-1 to 1)
        mosaic_clipped = mosaic_clipped.where((mosaic_clipped >= -1) & (mosaic_clipped <= 1), float("nan"))

        # store the clipped mosaic with the key as the acquistion date (DOY)
        clipped_evis[date] = mosaic_clipped

**NB**: We will now overlay the clipped EVI rasters for each half-month period, selecting the maximum pixel values to generate half-monthly EVI rasters. This process helps minimize the influence of cloud cover. Since the shapes (numbers of rows and columns) of the clipped EVI rasters differ, we need to standardize them to the same raster size before performing the overlay  

In [ ]:
# we need the bbox of GA_90 in meters (UTM) to calculate the raster size of the target raster grid for storing the clipped EVI files, so
# reproject the GA_90 from WGS84 to UTM 
# NB: we need the projected GA_90 just to calculate the target raster size, after that, we go back to working with the GA_90 in WGS84
crs_schema= CRS.from_epsg(32617)
GA_90_proj = GA_90.to_crs(crs_schema)
GA_90_proj_bbox= tuple(GA_90_proj.total_bounds)

# calculate the size of the raster
width = int((GA_90_proj_bbox[2] - GA_90_proj_bbox[0]) / 30), # no of rows
height = int((GA_90_proj_bbox[3] - GA_90_proj_bbox[1]) / 30) # no of columns


# define the target grid based on GA_90 bounding box
x_coords = np.linspace(GA_90.total_bounds[0], GA_90.total_bounds[2], width) 
y_coords = np.linspace(GA_90.total_bounds[1], GA_90.total_bounds[3], height)

# align and pad each clipped EVI to the target grid
aligned_evis = {}
for date, mosaic_data in clipped_evis.items():
    # align the clipped data to the target grid
    aligned_evi = mosaic_data.reindex(
        x=x_coords, 
        y=y_coords, 
        method="nearest",
        fill_value=float("nan")  # fill empty portions with NaN
    )
    # store the aligned EVI with the key as the acquistion date (DOY)
    aligned_evis[date] = aligned_evi

# aligned_evis.keys()

del(clipped_evis) # use this to free your memory

In [2]:
# --------------------------------
# group the EVI data in intervals of 16 days (half-month) and select the maximum pixels to create half monthly EVIs
# --------------------------------

# sort the aligned EVIs using their dates
julian_dates = sorted(aligned_evis.keys())

# Define the start DOY for each month 
start_date = 60  # March 1st in Julian date
# start_date = 152  # June 1st in Julian date
# start_date = 244  # September 1st in Julian date


interval = 16 # 16-day interval
grouped_evi = {}  # dictionary to store grouped EVI data arrays
interval_dates = []  # list to track interval start dates, we will need this later for the csv files
current_group = []  # temporary list for EVI data arrays in the current interval

# group the aligned EVIs based on intervals
for date in julian_dates:
    if date < start_date:
        continue  # skip dates before the specified start date
    if date < start_date + interval:
        current_group.append(aligned_evis[date])  # collect EVI data that fall within the interval
    else:
        grouped_evi[start_date] = xr.concat(current_group, dim="time")  # overlay EVIs within the same interval
        interval_dates.append(start_date)  # track the interval start date
        current_group = [aligned_evis[date]]  # start a new group
        start_date += interval

# combine any remaining EVI data into the last group
if current_group:
    grouped_evi[start_date] = xr.concat(current_group, dim="time")
    interval_dates.append(start_date)

# calculate maximum EVI for each group
max_evi_rasters = {
    start_date: group.max(dim="time", skipna=True)
    for start_date, group in grouped_evi.items()
}

# use the cotton farms polygon data to mask/clip the maximum EVI rasters
masked_max_evi_rasters = {}
for start_date, max_evi_da in max_evi_rasters.items():
    geometries = [feature for feature in GA_90_farms['geometry']]  # extract geometries to clip
    masked_max_evi = max_evi_da.rio.clip(geometries, GA_90_farms.crs, drop=True)  # clip data

    # store the masked EVI with the key as the acquistion date (DOY)
    masked_max_evi_rasters[start_date] = masked_max_evi

# average the EVI pixels in the clipped raster to get the 16-day composite evi per district
dist_evi = {
    start_date: masked_max_evi.mean().item()
    for start_date, masked_max_evi in masked_max_evi_rasters.items()
}

# create a DataFrame for the 16-day composite EVI values
GA_90_2015 = pd.DataFrame.from_dict(county_evi_means, orient='index', columns=['EVI'])

# export the DataFrame as a CSV file
output_path = os.path.join(final_dir, 'GA_90_2015_EVI_03_to_05.csv') # use the months that was specified at the start in the filename
GA_90_2015.to_csv(output_path, index=True)

After running the code till this point, delete all the files in your 'Image_files' folder to free up storage space for the images of the rest of the months and years in the district.
Then, go back to the start to change the start and end date for searching the HLS data and run the code again till you get all the 16-day composite EVI for each year (2015 to 2024).

### PART B
- This is where we will merge all the csv files and select the maximum EVI per month to get the monthly EVI from March to November of 2015 to 2024.   
 
**NB:** Unlike part A, this very straightforward. 

In [ ]:
# change the base directory to the EVI_CSV folder
path= os.chdir('C:/TX_GA_CSB/per_district/EVI_CSV')

# load all the csv files
GA90_files= glob.glob('*csv')

# initialize an empty DataFrame to store data for all years
GA90_years = pd.DataFrame()

for year in range(2015, 2025):
    merged_df = pd.DataFrame() # temporary DataFrame for the current year

    # filter data for each year and sort the files in ascending order
    GA90_files_sorted = sorted([file for file in GA90_files if f"{year}" in file])

    # read each csv file and append as a DataFrame and append to the merged DataFrame
    for file in GA90_files_sorted:
        merged_df = pd.concat([merged_df, pd.read_csv(file)], ignore_index=True)
            
    merged_df.rename(columns={merged_df.columns[0]: 'Sub_month'}, inplace=True) # rename the first column (which is the start DOY of 16-day interval)

    # replace the Sub_month values with alternating 1 and 2, which represents first and second half of a month respectively
    merged_df['Sub_month'] = [1 if i % 2 == 0 else 2 for i in range(len(merged_df))]

    # create a 'Month' column with values from 3 to 11, repeating each value twice
    months = [month for month in range(3, 12) for _ in range(2)]  # each month repeats twice
    merged_df['Month'] = months[:len(merged_df)]  # ensure the length matches the DataFrame
    merged_df['Year'] = year  # add the current year as a new column

    # append the merged DataFrame for this year to the final DataFrame
    GA90_years = pd.concat([GA90_years, merged_df], ignore_index=True)

# select the maximum EVI value for each month to get monthly EVI from 2015 to 2024
GA90_max= GA90_years.loc[GA90_years.groupby(['Year', 'Month'])['EVI'].idxmax()].reset_index(drop= True)
GA90_max.drop(columns=['Sub_month'], inplace= True) # drop the Sub_month column

# save as csv file
GA90_max.to_csv('GA_90_Monthly_EVI_2015_to_2024.csv')